# 1. 环境配置

## 1.1 python 环境准备

In [1]:
! pip install openai==2.11.0 dashscope==1.25.4 langchain-classic==1.0.0 langchain==1.1.3 langchain-community==0.4.1 langchain-openai==1.1.3 arxiv==2.3.1

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


## 1.2 大模型密钥准备

请根据第一章内容获取相关平台的 API KEY，如若未在系统变量中填入，请将 API_KEY 信息写入以下代码（若已设置请忽略）：

In [2]:
import os

# os.environ["OPENAI_API_KEY"] = "sk-xxxxxxxx"
# os.environ["DASHSCOPE_API_KEY"] = "sk-yyyyyyyy"

## 1.3 实践代码

为了能够顺利的演示内置中间件的使用详情，这里我们使用一段简单的智能体代码演示：

In [3]:
from langchain_community.chat_models import ChatTongyi
import os
llm = ChatTongyi(api_key=os.environ.get("DASHSCOPE_API_KEY"), model="qwen-turbo")

from langchain_community.agent_toolkits.load_tools import load_tools
tools = load_tools(["arxiv"])

from langgraph.checkpoint.memory import InMemorySaver 
memory = InMemorySaver()

from langchain.agents import create_agent
agent = create_agent(model=llm, 
                     tools=tools, 
                     system_prompt="You are a helpful assistant", 
                     checkpointer=memory)

result1 = agent.invoke({"messages": [{"role": "user", "content": "请使用 arxiv 工具查询论文编号 1605.08386"}]}, config={"configurable": {"thread_id": "user_1"}})
print(result1["messages"][-1].content)

论文编号 1605.08386 的信息如下：

- **发表日期**: 2016-05-26
- **标题**: Heat-bath random walks with Markov bases
- **作者**: Caprice Stanley, Tobias Windisch
- **摘要**: 研究了格点上的图，其边来自于有限集的允许移动。我们证明了这些图在固定整数矩阵的纤维上的直径可以被常数上界限制。然后研究了这些图上的热浴随机游走的混合行为。还给出了移动集的显式条件，使得热浴随机游走（Glauber动力学的一种推广）在固定维度中是扩展器。


# 2. 内置中间件

## 2.1 简介

针对于一些智能体开发常用的中间件，LangChain 官方团队给出了一系列的内置中间件来帮助大家进行快速上手。这些中间件包括：
- ModelFallbackMiddleware：主模型失败时自动换模型
- ModelCallLimitMiddleware：限制模型调用次数
- ToolCallLimitMiddleware：限制工具调用频率
- ToolRetryMiddleware：工具失败时自动重试
- LLMToolEmulator：用LLM模拟工具执行
- SummarizationMiddleware：摘要对话历史
- ContextEditingMiddleware：清理旧上下文
- PIIMiddleware：检测/脱敏个人隐私信息
- HumanInTheLoopMiddleware：人工审批确认机制
- ...

## 2.2 ModelFallbackMiddleware：模型容灾与降级策略

在实际部署智能体（Agent）时，经常会遇到这些情况：
- 主模型（如 qwen-max）暂时不可用或返回错误；
- 模型超时、API 限额、或延迟太高；
- 想节省成本，用小模型在轻任务中“兜底”。

所以 LangChain 官方提供了一个作用在 wrap_model_call 的内置中间件 ModelFallbackMiddleware ，在主模型失败时自动切换备用模型，保持任务不中断。

In [4]:
from langchain.agents.middleware import ModelFallbackMiddleware

agent = create_agent(
  model=ChatTongyi(model="xxx"), # 错误模型
  tools=tools,
  system_prompt="You are a helpful assistant",
  middleware=[ModelFallbackMiddleware(
      ChatTongyi(model="qwen-plus"),
      ChatTongyi(model="qwen-turbo",
      checkpointer=memory))])

result1 = agent.invoke({"messages": [{"role": "user", "content": "请使用 arxiv 工具查询论文编号 1605.08386"}]}, config={"configurable": {"thread_id": "user_1"}})

print(result1["messages"][-1].content)

根据您查询的论文编号 1605.08386，以下是相关信息：

**发表时间**: 2016-05-26  
**标题**: Heat-bath random walks with Markov bases (使用马尔可夫基的热浴随机游走)  
**作者**: Caprice Stanley, Tobias Windisch  

**摘要**: 研究了格点上具有来自有限组任意长度允许移动的边的图。我们证明了这些图在固定整数矩阵纤维上的直径可以从上方由一个常数限定。然后我们研究了这些图上热浴随机游走的混合行为。我们还给出了移动集合的明确条件，使得热浴随机游走（Glauber动力学的一种推广）在固定维度下是一个扩张器。

这篇论文主要研究了基于马尔可夫基的热浴随机游走的数学性质，特别是其在特定图结构上的混合行为和扩张性。


## 2.3 ModelCallLimitMiddleware：限制模型调用次数

在一个智能体执行循环（Agent Loop）中，模型会多次被调用：
- 一次决定“要不要用工具”；
- 一次执行后判断“是否需要再次调用”；
- 再次思考结果、继续调用或返回最终结果。

在 ReAct-style agent 中，如果 LLM 的输出不够理性，可能会进入：输入→ 模型思考 → 工具调用 → 模型思考 → 工具调用 → 模型思考... 这种循环，轻则多花钱，重则“跑爆上下文”。

因此，ModelCallLimitMiddleware 的作用就是限定模型调用次数，一旦达到上限，就让 Agent 自动停止或报错。

In [ ]:
from langchain.agents.middleware import ModelCallLimitMiddleware

middleware_model_limit = ModelCallLimitMiddleware(thread_limit=2, run_limit=1)

agent = create_agent(
    model=llm,
    tools=tools,
    system_prompt="You are a helpful assistant",
    middleware=[middleware_model_limit],
    checkpointer=InMemorySaver()
)

config3 = {"configurable": {"thread_id": "user_3"}}

result1 = agent.invoke({"messages": [{"role": "user", "content": "请使用 arxiv 工具查询论文编号 1605.08386，100字即可"}]}, config=config3)
result2 = agent.invoke({"messages": [{"role": "user", "content": "你能总结一下吗？100字即可"}]}, config=config3)
result3 = agent.invoke({"messages": [{"role": "user", "content": "推荐几篇拓展阅读吧"}]}, config=config3)

print(result3["messages"][-1].content)

Model call limits exceeded: thread limit (2/2)


## 2.4 ToolCallLimitMiddleware：限制工具调用次数
类似的，除了有模型调用的限制以外，还有一个就是限制工具调用的次数。在实际智能体（Agent）执行循环中，工具调用（Tool Calls）往往是最昂贵或最不确定的环节：
- 有的工具访问外部 API（比如搜索、数据库查询）；
- 有的工具运行时间长（比如爬虫、OCR、代码执行）；
- 如果模型逻辑出错，可能疯狂调用工具，导致成本暴涨或资源占满。

这也是为什么 LangChain 提供了 ToolCallLimitMiddleware ，通过在调用工具的前后设置调用次数上限，防止 Agent 过度调用工具或陷入无穷循环。
- before_model：判断调用次数是否达到上限；
- after_model：统计这次模型调用中生成了几个 tool_call 并计数。

In [6]:
from langchain.agents.middleware import ToolCallLimitMiddleware

middleware=[
    ToolCallLimitMiddleware(run_limit=10),  # 全局
    ToolCallLimitMiddleware(tool_name="arxiv", thread_limit=1),  # 局部
]

agent = create_agent(
    model=llm,
    tools=tools,
    system_prompt="You are a helpful assistant",
    middleware=middleware,
    checkpointer=InMemorySaver()
)

config3 = {"configurable": {"thread_id": "user_3"}}

result1 = agent.invoke({"messages": [{"role": "user", "content": "请使用 arxiv 工具查询论文编号 1605.08386，100字即可"}]}, config=config3)
result2 = agent.invoke({"messages": [{"role": "user", "content": "请使用 arxiv 工具查询论文编号 2103.08386，100字即可"}]}, config=config3)


print(result2["messages"][-2].content)


Tool call limit exceeded. Do not call 'arxiv' again.


## 2.5 ToolRetryMiddleware：工具调用自动重试机制

在工具调用的过程中，我们常常还会遇到一些外部的问题，比如网络连接失败、数据库连接失败、模型执行有问题等等。那在工具调用（Tool call）失败时，如何让 Agent 不会立即崩溃或返回错误，这就成了急切需要解决的问题。

所以这个时候，ToolRetryMiddleware 就给出了一个方案，不仅仅可以在调用工具的时候重复多次，还能够设置延时、指数回退（exponential backoff）、随机扰动（jitter）等机制。从而让智能体在调用不稳定外部服务（例如网络请求、搜索接口、数据库API）时更加鲁棒（robust）。

其作用的位置是在 wrap_tool_call 中，也就是工具调用时。其目标就是如果工具出错，就重试几次。

In [7]:
from langchain.agents import create_agent
from langchain.agents.middleware import ToolRetryMiddleware

agent = create_agent(
  model=llm,
  tools=tools,
  system_prompt="You are a helpful assistant",
  middleware=[
    ToolRetryMiddleware(
      max_retries=3,   # 最多重试 3 次
      backoff_factor=2.0, # 指数退避倍数
      initial_delay=1.0, # 首次延迟 1 秒
      max_delay=60.0,   # 最大等待时间 60 秒
      jitter=True,    # 添加随机扰动，防止雪崩重试
    ),
  ],
  checkpointer=memory)

## 2.6 LLMToolEmulator：用模型“假装”执行工具
有些时候，在还没有实现具体的工具逻辑的时候，但是这个工具又非常重要的时候，我们可以通过用 LLM 来模拟工具的执行结果（tool emulation）。这样就可以让整体流程先顺利进行，后面再补充相对应的逻辑。

比如当 agent 决定调用某个工具时（如 search_database()），这个中间件会拦截掉真正的工具调用，然后由另一个 LLM 生成“模拟的工具返回结果”。


In [8]:
from langchain.agents.middleware import LLMToolEmulator

agent = create_agent(
    model=llm,
    tools=tools,
    middleware=[
        LLMToolEmulator(
            model = ChatTongyi(model="qwen-turbo"),
            tools=["arxiv"],
        ), 
    ],
)

result1 = agent.invoke({"messages": [{"role": "user", "content": "请使用 arxiv 工具查询论文编号 1605.08386"}]}, config={"configurable": {"thread_id": "user_1"}})
print(result1["messages"][-1].content)

论文编号 1605.08386 的标题是《The Nonlinear Diffusion Equation》，作者是 J. M. Burgers。该论文发表于 1960 年 1 月 1 日，属于“Mathematical Physics”领域。摘要中提到，该论文详细研究了非线性扩散方程，重点分析了其数学结构和物理意义，并在流体力学和热传递的背景下探讨了冲击波的形成以及在不同初始条件下的解的行为。

你可以通过 [此链接](https://arxiv.org/pdf/1605.08386.pdf) 查看或下载这篇论文的 PDF 版本。


当我们希望所有的工具都用 LLM 进行模拟，我们可以这样写入（tools 参数里传入 None）：

In [ ]:
from langchain.agents.middleware import LLMToolEmulator
from langchain.tools import tool

@tool
def get_weather(location: str) -> str:
    ''' 获取指定位置的天气信息 '''
    message = f"获取 {location} 的天气信息"
    print(message)  
    return message

@tool
def search_database(query: str) -> str:
    ''' 从数据库中搜索相关信息 '''
    message = f"数据库中关于 {query} 的信息是：..."
    print(message)  
    return message  
    
@tool
def send_email(to: str, subject: str, body: str) -> str:
    ''' 发送邮件 '''
    message = f"已发送邮件至 {to}，主题为 {subject}，内容为 {body}"
    print(message)  
    return message

agent = create_agent(
    model=llm,
    tools=[get_weather, search_database, send_email],
    middleware=[
        LLMToolEmulator(
            model = ChatTongyi(model="qwen-turbo")
        ),  # 所有工具调用都由 LLM 模拟执行
    ],
)

# 调用时，模型不会真的去查数据库或发邮件
result = agent.invoke({"messages": [{"role": "user", "content": "查一下广州的天气"}]})

print(result["messages"][-1].content)


广州的天气是多云，气温28摄氏度，湿度65%，风速10公里每小时。


## 2.7 LLMToolSelectorMiddleware：智能工具筛选器

当一个 Agent 拥有很多工具（10 个以上）时，模型每次推理都需要在所有工具说明之间“思考”，这样不仅浪费 token，还容易选错。

所以 LLMToolSelectorMiddleware 的作用就是在主模型调用前，让一个小模型（或更快的模型）先分析用户意图 → 从众多工具中选出相关的几个。

最终，Agent 的系统提示中只保留这些“相关工具”的定义，从而实现 更快、更准、更省钱 的调用。这在一定程度上也算是 Context Engineering （上下文工程）的实践。

In [10]:
from langchain.agents.middleware import LLMToolSelectorMiddleware

agent = create_agent(
    model=llm,
    tools=[get_weather, search_database, send_email],
    middleware=[
        LLMToolSelectorMiddleware(
            model=ChatTongyi(model="qwen-turbo"),  # ✅ 选择辅助模型用于筛选
            max_tools=2,                 # 最多保留 2 个
            always_include=["send_email"],   # 某些关键工具始终保留
        ),
    ],
)

result = agent.invoke({"messages": [{"role": "user", "content": "查一下广州的天气"}]})

print(result["messages"][-1].content)

获取 广州 的天气信息
广州今天的天气是晴天，气温在25°C到32°C之间，风力较小，适合外出活动。


## 2.8 TodoListMiddleware：任务拆解与规划能力
在复杂任务中，模型常出现：
- 一步把所有事情混在一起做
- 任务顺序混乱
- 忘记步骤
- 无法跟踪进度
- 无法形成一个持久化的任务列表

这类问题在大型项目（写代码、调试、多文件编辑）中尤为明显。

因此 LangChain 提供 TodoListMiddleware，让智能体内部自动拥有“规划→执行→更新”的能力。

In [11]:
from langchain.agents.middleware import TodoListMiddleware

agent = create_agent(
    model=llm,
    tools=tools,
    middleware=[TodoListMiddleware()],
)

task = '''请帮我完成以下任务：

1. 使用 arxiv 搜索论文 1605.08386，并提取主要贡献。
2. 搜索论文 1605.08387，并比较两篇论文的研究主题差异。
3. 最后写一个 300 字总结，说明它们在研究方法上的共同点。

请分步骤执行，并进行规划。'''

result1 = agent.invoke({"messages": [{"role": "user", "content": f"{task}"}]}, config={"configurable": {"thread_id": "user_1"}})
print(result1)

{'messages': [HumanMessage(content='请帮我完成以下任务：\n\n1. 使用 arxiv 搜索论文 1605.08386，并提取主要贡献。\n2. 搜索论文 1605.08387，并比较两篇论文的研究主题差异。\n3. 最后写一个 300 字总结，说明它们在研究方法上的共同点。\n\n请分步骤执行，并进行规划。', additional_kwargs={}, response_metadata={}, id='bcfba9ed-7878-48a3-99fc-787bf1e1dd95'), AIMessage(content='', additional_kwargs={'tool_calls': [{'function': {'arguments': '{"todos": [{"content": "使用 arxiv 搜索论文 1605.08386，并提取主要贡献。", "status": "in_progress"}, {"content": "搜索论文 1605.08387，并比较两篇论文的研究主题差异。", "status": "pending"}, {"content": "最后写一个 300 字总结，说明它们在研究方法上的共同点。", "status": "pending"}]}', 'name': 'write_todos'}, 'id': 'call_94d704fa7743401b9c38a2', 'index': 0, 'type': 'function'}]}, response_metadata={'model_name': 'qwen-turbo', 'finish_reason': 'tool_calls', 'request_id': '1d8e3e7d-232c-4579-b5fd-9965696c0a40', 'token_usage': {'input_tokens': 1437, 'output_tokens': 116, 'prompt_tokens_details': {'cached_tokens': 0}, 'total_tokens': 1553}}, id='lc_run--019b220e-2a0c-7b60-9679-57d01a53f494-0', tool_calls=[{'n